# Day 4 - claude

In [1]:
def read_grid(filename):
    """Read the input file and return the grid as a list of strings."""
    with open(filename) as f:
        return [line.strip() for line in f if line.strip()]

def get_directions():
    """Return all possible directions to search for words."""
    return [
        (0, 1),   # right
        (1, 0),   # down
        (1, 1),   # diagonal down-right
        (-1, 1),  # diagonal up-right
        (0, -1),  # left
        (-1, 0),  # up
        (-1, -1), # diagonal up-left
        (1, -1)   # diagonal down-left
    ]

def check_word(grid, row, col, dx, dy, word):
    """Check if the word exists starting from (row, col) in direction (dx, dy)."""
    if not (0 <= row < len(grid) and 0 <= col < len(grid[0])):
        return False
        
    # Check if we can fit the word in this direction
    if not (0 <= row + dx * (len(word)-1) < len(grid) and 
            0 <= col + dy * (len(word)-1) < len(grid[0])):
        return False
    
    # Check each character
    for i in range(len(word)):
        if grid[row + dx * i][col + dy * i] != word[i]:
            return False
    return True

def find_all_occurrences(grid, word):
    """Find all occurrences of the word in the grid in all directions."""
    occurrences = []
    directions = get_directions()
    
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            for dx, dy in directions:
                if check_word(grid, i, j, dx, dy, word):
                    occurrences.append((i, j, dx, dy))
    
    return occurrences

def create_marked_grid(grid, word, occurrences):
    """Create a new grid with only the found word occurrences marked."""
    rows = len(grid)
    cols = len(grid[0])
    marked = [['.' for _ in range(cols)] for _ in range(rows)]
    
    # Mark each occurrence
    for row, col, dx, dy in occurrences:
        for i in range(len(word)):
            marked[row + dx * i][col + dy * i] = grid[row + dx * i][col + dy * i]
    
    return [''.join(row) for row in marked]

# Read the input grid
grid = read_grid('input.txt')
word = 'XMAS'

# Find all occurrences
occurrences = find_all_occurrences(grid, word)

# Print the total count
print(f"'{word}' appears {len(occurrences)} times in the grid.")

# Create and print the marked grid
marked_grid = create_marked_grid(grid, word, occurrences)
print("\nMarked grid (only showing letters involved in XMAS):")
for row in marked_grid:
    print(row)

'XMAS' appears 2464 times in the grid.

Marked grid (only showing letters involved in XMAS):
...XX..X.....................S.....SAMX.S...S.S...S........S...S....SAMX...........XSAMX.......S.S.S.S...........S..S.....XMAS............S.
..MMM.M.XMAS....SAMXMASAMX..XSA....SAMXXA.X.A.A...AX.XMAS..SA...AS...A.........X....MAMX.SAMX..AA.A.A............AXA...........X...SAMX..A..
.AA.AA...........XMAS...S...AM.M....A.M.MSM.M.M...M.XXMX....AM...MAS..M..SS...S.M...SAM.....M..MSMSMS...SSX.....XMMX..........M........XMAS.
SS.XS.S...S..........A.A...MXMASX..S..A.XAA.X.X..AXSMMMAM....MX.XXXMA..XA..A.A.A.A.XSASX....A.XXXAXAX...AA.MS..M.XAM.........A........MX.X..
...XM....X.A.X........M...XMAS.S.M.S..SSMMS..X..S.A.AA..SA....XMMMMSXM.M....M...M.SMS.M.....S.S.M.M.....MM..A.A.SMSA.....XMAS...S.S..A...M..
S.M.SA..MMS.M........X.X........X.A.A.A.AX...XMASM..SSS.S.SS..A.AAAXA.XA...X.X...X.AX..X.SS...AX.X.X....XX..MS..AA.S......S....A.A.AS..X.A..
.A..A.SA.A.A.X...X..XMAS.........M.S.M..S......AX...XAA.XAXXA

## Part 2

In [ ]:
def read_grid(filename):
    """Read the input file and return the grid as a list of strings."""
    with open(filename) as f:
        return [line.strip() for line in f if line.strip()]

def check_mas(grid, row, col, dr1, dc1, dr2, dc2):
    """
    Check if there are two valid MAS patterns starting at (row, col)
    going in directions (dr1, dc1) and (dr2, dc2).
    MAS can be forwards or backwards (SAM).
    """
    # Check bounds
    if not (0 <= row < len(grid) and 0 <= col < len(grid[0])):
        return False
    
    # Check if we can fit both patterns
    if not (0 <= row + dr1 * 2 < len(grid) and 
            0 <= col + dc1 * 2 < len(grid[0]) and
            0 <= row + dr2 * 2 < len(grid) and 
            0 <= col + dc2 * 2 < len(grid[0])):
        return False
    
    # Get the two potential MAS strings
    str1 = grid[row][col] + grid[row + dr1][col + dc1] + grid[row + dr1 * 2][col + dc1 * 2]
    str2 = grid[row][col] + grid[row + dr2][col + dc2] + grid[row + dr2 * 2][col + dc2 * 2]
    
    # Check if both strings are either MAS or SAM
    valid_patterns = {'MAS', 'SAM'}
    return str1 in valid_patterns and str2 in valid_patterns

def find_x_mas_patterns(grid):
    """Find all X-MAS patterns in the grid."""
    patterns = set()  # Using set to avoid duplicates
    rows = len(grid)
    cols = len(grid[0])
    
    # Only the valid X shape direction pairs
    x_patterns = [
        ((1, 1), (1, -1)),    # down-right and down-left
        ((1, -1), (1, 1)),    # down-left and down-right
    ]
    
    for i in range(rows):
        for j in range(cols):
            for (dr1, dc1), (dr2, dc2) in x_patterns:
                if check_mas(grid, i, j, dr1, dc1, dr2, dc2):
                    patterns.add((i, j))
    
    return patterns

def create_marked_grid(grid, patterns):
    """Create a new grid marking only the X-MAS patterns."""
    rows = len(grid)
    cols = len(grid[0])
    marked = [['.' for _ in range(cols)] for _ in range(rows)]
    
    # X shape directions
    directions = [(1, 1), (1, -1)]  # down-right and down-left
    
    # Mark each pattern
    for row, col in patterns:
        # Mark the center point
        marked[row][col] = grid[row][col]
        # Mark both diagonals
        for dr, dc in directions:
            for i in range(1, 3):
                r, c = row + dr * i, col + dc * i
                if 0 <= r < rows and 0 <= c < cols:
                    marked[r][c] = grid[r][c]
    
    return [''.join(row) for row in marked]

# Read the input grid
grid = read_grid('input.txt')

# Find all X-MAS patterns
patterns = find_x_mas_patterns(grid)

# Print the total count
print(f"Found {len(patterns)} X-MAS patterns in the grid.")

# Create and print the marked grid
marked_grid = create_marked_grid(grid, patterns)
print("\nMarked grid (only showing letters involved in X-MAS patterns):")
for row in marked_grid:
    print(row)

Found 822 X-MAS patterns in the grid.

Marked grid (only showing letters involved in X-MAS patterns):
................S.....M...................S.S.S.S..........S.M.S.M..............................MS.S.S......................................
...............A.A...A.A...........S.....A.A.A.A.A........A.A.A.A.A............................AAAA.A.A......S..................S...........
..............M...M.S.M.S.........A.A...M.M.M.M.M.M......M.S.M.S.M.S........S.S...............SM.MSM.M.M....A.A...............MA.A..........
.........S.S.........A.A.........M.S.M.A.A.................................A.A.A...........................M.S.M......S......AMA..M.........
........A.A.A.......S...S.........ASA.S...S....................M.M......S.M.M.M.M...S.......................A.A......A.A....S...S...........
.......MMMSM.M...................MA.AM..................S.....A.A.A....A.A....M....A.A....S.......S........M...M....M...M..A.A.A.A..........
.......A.A.A................S....M...M....S.........

: 